In [1]:
import pickle
import os

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline

### Data Prerequisites

In [2]:
for data_ in ['2021-01', '2021-02']:
    data_path = 'data/green_tripdata_'+data_+'.parquet'
    if not os.path.exists(data_path):
        url = f"https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_{data_}.parquet"
        try:
            os.system(f"wget {url} -P \data")
        except:
            print(f"{url} not available")

--2022-06-17 18:14:41--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.9.214
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.9.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1,3M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2021-01.parquet’

     0K .......... .......... .......... .......... ..........  3%  227K 6s
    50K .......... .......... .......... .......... ..........  7%  437K 4s
   100K .......... .......... .......... .......... .......... 11% 45,3M 3s
   150K .......... .......... .......... .......... .......... 15%  117M 2s
   200K .......... .......... .......... .......... .......... 19%  448K 2s
   250K .......... .......... .......... .......... .......... 23% 53,9M 2s
   300K .......... .......... .......... .......... .......... 26% 51,0M 1s
   350K .......... .......... .......... .......... .......... 30% 65,1M 1s
   400K

### Set MLflow local tracking system

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("green-taxi-duration")

2022/06/17 18:14:58 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/06/17 18:14:58 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='green-taxi-duration', tags={}>

### Train the model

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


### Compare with what is online

In [29]:
pipeline

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, min_samples_leaf=10,
                                       n_jobs=-1, random_state=0))])

In [30]:
ride = {
    "PULocationID": 10,
    "DOLocationID": 50,
    "trip_distance": 40, 
}

In [31]:
import predict
features = predict.prepare_features(ride)
pred = predict.predict(features)

In [32]:
pred

45.50965007660852

In [33]:
! python test.py

{'duration': 45.50965007660852}
